# **1. Import Libraries**


In [1]:
import pandas as pd
from datetime import date, timedelta
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import math
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Load Data 

This population data is pulled from the 2019 US Census and is here along with latitude and longitude data for each states capital city:

In [2]:


# Population Data
populationData = pd.read_csv('/kaggle/input/2019-census-us-population-data-by-state/2019_Census_US_Population_Data_By_State_Lat_Long.csv')

# vaccintion data
vaccinationData = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv')


In [3]:
# filtering vaccination dates where date is 10th of may 2023 

# Vaccination data, for most recent date
vaccinationByLocation = vaccinationData.loc[(vaccinationData.date == '2023-05-10')][["location", "people_vaccinated"]]

In [4]:
# merge vaccination data with population data 
# Vaccination and population data
vaccinationAndPopulationByLocation = pd.merge(populationData, vaccinationByLocation, left_on='STATE',right_on='location').drop(columns="location")


In [5]:
# Calculate percentage vaccinated persons by state
vaccinationAndPopulationByLocation["percent_vaccinated"] = vaccinationAndPopulationByLocation["people_vaccinated"] / vaccinationAndPopulationByLocation["POPESTIMATE2019"]

vaccinationAndPopulationByLocation

,STATE,POPESTIMATE2019,lat,long,people_vaccinated,percent_vaccinated
0,Alabama,4903185,32.377716,-86.300568,3193141.0,0.651238
1,Alaska,731545,58.301598,-134.420212,535718.0,0.732310
2,Arizona,7278717,33.448143,-112.096962,5704677.0,0.783748
3,Arkansas,3017804,34.746613,-92.288986,2115165.0,0.700895
4,California,39512223,38.576668,-121.493629,33613401.0,0.850709
5,Colorado,5758736,39.739227,-104.984856,4837792.0,0.840079
6,Connecticut,3565287,41.764046,-72.682198,3670090.0,1.029395
7,Delaware,973764,39.157307,-75.519722,861811.0,0.885031
8,District of Columbia,705749,38.895110,-77.036370,836680.0,1.185521
9,Florida,21477737,30.438118,-84.281296,17810446.0,0.829252


In [6]:
print("Date ran:", date.today())

# Calculate the total percent vaccinated in the US
percentageTotal = vaccinationAndPopulationByLocation["people_vaccinated"].sum() / vaccinationAndPopulationByLocation["POPESTIMATE2019"].sum()
print('Percentage Vaccinated in the US: {}%'.format(round(percentageTotal*100, 2))) 

Date ran: 2023-12-08
Percentage Vaccinated in the US: 80.04%


# 3. Data Tracker

We create a tracker using the vaccination data, with one marker representing each state.  Click on the markers to see the percentage of the population that has been vaccinated.

In [7]:
# Create the map
v_map = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=4) 

In [8]:
# Display the map
v_map

In [9]:
# Add points to the map
mc = MarkerCluster()
for idx, row in vaccinationAndPopulationByLocation.iterrows(): 
    if not math.isnan(row['long']) and not math.isnan(row['lat']):
        mc.add_child(Marker(location=[row['lat'], row['long']],
                            tooltip=str(round(row['percent_vaccinated']*100, 2))+"%"))
v_map.add_child(mc)

# 4. Bubble Maps

Bubble Maps uses circle instead of markers, we can use circles and colors to vary the size of the each circle.

In [10]:
 vaccinationAndPopulationByLocation.head()

,STATE,POPESTIMATE2019,lat,long,people_vaccinated,percent_vaccinated
0,Alabama,4903185,32.377716,-86.300568,3193141.0,0.651238
1,Alaska,731545,58.301598,-134.420212,535718.0,0.732310
2,Arizona,7278717,33.448143,-112.096962,5704677.0,0.783748
3,Arkansas,3017804,34.746613,-92.288986,2115165.0,0.700895
4,California,39512223,38.576668,-121.493629,33613401.0,0.850709


In [11]:
b_map = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=4) 

Here we create color producer: if percent is less, than 0.5, it will be colored in green, if value is between [0.5; 0.7) it will be in orange, otherwise in red
.

In [12]:
def color_producer(value):
    if value < 0.5:
        return 'green'
    elif (0.5 <= value < 0.7):
        return 'darkorange'
    else:
        return 'red'
    

In [13]:
# add bubble map
for i in range(0, len(vaccinationAndPopulationByLocation)):
    Circle(
        location= [vaccinationAndPopulationByLocation.iloc[i]['lat'],vaccinationAndPopulationByLocation.iloc[i]['long']],
        radius=float(vaccinationAndPopulationByLocation.iloc[i]['percent_vaccinated'])*1000,
        fill = True,
        color=color_producer(vaccinationAndPopulationByLocation.iloc[i]['percent_vaccinated'])
        
    ).add_to(b_map)
    
b_map

# 5. HeatMap
This shows the density of vaccinated people in different areas of the city, where red areas have relatively more vaccinated people.

In [14]:
h_3 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=4) 

HeatMap(data=vaccinationAndPopulationByLocation[['lat', 'long']], radius=10).add_to(h_3)
h_3

#  6. Conclusion

The End. Thank you all for looking through my analysis. 

Kindly drop any comments and upvote!!